In [1]:
import matplotlib.pyplot as plt
import os

import numpy as np

from project_classes import *
from functions import *
# from iceflow_library import *
# import scipy.optimize as optimize
# from scipy.optimize import curve_fit
print("Imports successful.")

Imports successful.


In [2]:
zoom = True
seg_length = 250
# season = "2009_Antarctica_DC8"
# season1 = "2018_Antarctica_DC8" 
# season2 = "2018_Antarctica_DC8" 
# season1 = "2016_Antarctica_DC8" 
# season2 = "2016_Antarctica_DC8" 
# season = "2014_Antarctica_DC8" 
# season1 = "2022_Antarctica_BaslerMKB"
season = "2022_Antarctica_BaslerMKB"
# season1 = "2013_Antarctica_P3"

# file_name1 = "C:\\Users\\moser\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + flight1 + ".pickle"
# file_name2 = "C:\\Users\\moser\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + flight2 + ".pickle"
testing = False

In [15]:
def mat_pickler_h5py(season, flight, testing_mode=False, readout=False, save=True, plot_layer=False, convert_xy=False):
    # print("Reading data files...")
    # print(section_break)
    # set the directory, segment data file, layer attributes file, and start and end frames
    if testing_mode:
        # the dir is the current directory + the test_data folder
        dir = os.getcwd() + '\\test_data\\' + flight + '\\'
    else:
        # TODO: refactor this into a try except block, maybe upstream of this function where it is called
        dir = ('C:\\Users\\moser\\Documents\\cresis\\rds\\' + season + '\\CSARP_layer\\' + flight + '\\')

    data_file = dir + 'Data_' + flight + '_'
    attributes_file = dir + 'layer_' + flight
    layers = layerize_h5py(data_file, attributes_file, dir, convert_xy)
    if layers == True:
        return True

    if readout:
        print(section_break, end="")
        for layer in layers:
            print(f"\n{layer.layer_name} number of points: {layer.twtt.shape[0]}")
            print(f"{layer.layer_name} twtt first three: {layer.twtt[:3].tolist()} ")
            print(f"{layer.layer_name} twtt last three: {layer.twtt[-3:].tolist()} ")
        print(section_break + "\n")

    if save:
        # save layers to a pickle file
        # print("Saving layers to a pickle file...")
        # print(section_break)
        # list current directory
        # print(f"Current directory: {}")
        directory = os.getcwd() + "\\pickle_jar\\"
        # file_name = directory + "layer_export" + attributes_file[5:-4] + ".pickle"
        file_name = directory + "layer_export_" + flight + ".pickle"
        pickle.dump(layers, open(file_name, "wb"))
        # print(file_name, " saved in local directory of this python file.")
        # print(section_break + "\n")

    if plot_layer:
        # plot the layers
        print("Plotting layers...")
        print(section_break)
        # plot the layer depths vs gps time for each layer on the same plot
        for layer in layers:
            plt.plot(layer.gps_time, layer.twtt, label=layer.layer_name)
        plt.xlabel("GPS Time")
        plt.ylabel("Two Way Travel Time (ns)")
        # invert the y axis so that the plot is right side up
        plt.gca().invert_yaxis()
        plt.title("Elevation vs GPS Time")
        plt.legend()

        plt.show()
        print(section_break + "\n")


def layerize_h5py(data_file, attribute_file, dir, convert_xy):
    # TODO: add a docstring
    # TODO: add a portion to pull the layer name out of the attribute file
    # afai can tell, that is all the attribute file is used for in the original code
    # create an empty list to store the layers
    layers = []
    files = os.listdir(dir)  # list of files in the directory
    # TODO: shift to differentiating data vs layer based on file name. Data starts with "Data_" and layer starts with "layer_"
    startframe = '001'  # the first file number in the directory
    endframe = str(len(files) - 1).zfill(3)  # the number of files in the directory
    # debug_print(f"attribute_file: {attribute_file}")
    attribute_mat = h5py.File(attribute_file + '.mat', 'r')
    # print the keys in the attribute file

    decimal1 = attribute_mat[np.array([attribute_mat['lyr_name'][0][0]])[0]]
    decimal1_name = [decimal1[i][0] for i in range(len(decimal1))]  # a list of unicode values of the characters
    decimal2 = attribute_mat[np.array([attribute_mat['lyr_name'][1][0]])[0]]
    decimal2_name = [decimal2[i][0] for i in range(len(decimal2))]  # a list of unicode values of the characters
    name1 = ''.join(chr(i) for i in decimal1_name)  # convert the unicode values to a string
    name2 = ''.join(chr(i) for i in decimal2_name)  # convert the unicode values to a string

    layer1_name = name1
    layer2_name = name2

    # print(f"data_file: {data_file}")
    filled_data_file = data_file + str(1).zfill(3) + '.mat'
    # print(f"filled_data_file: {filled_data_file}")

    h5py_works = True
    # debug_print("made it to layerize_h5py", "passed the filled_data_file variable to h5py.File",
    # "\n", "filled_data_file: ", filled_data_file, "\n")
    try:
        f = h5py.File(filled_data_file, 'r')
        debug_print(BRIGHT_GREEN, f"Opened {filled_data_file} with h5py")
    except:
        # debug_print(RED, f"Error: could not open {filled_data_file} with h5py, ignoring this file")
        # return True
        # print(f"Error: could not open {filled_data_file} with h5py, attempting with sio.loadmat")
        try:
            f = sio.loadmat(filled_data_file)
            h5py_works = False
        except:
            debug_print(RED, f"Error: could not open {filled_data_file} with sio.loadmat, exiting")
            # sys.exit(1)
            return True      


    # f = h5py.File(data_file + str(3).zfill(3) + '.mat', 'r')  #
    # print the contents of the 'param' key
    # print(f"f['param']: {f['param']}")
    # print(f"f['param'].keys(): {list(f['param'].keys())}")

    filename = data_file + '001.mat'  # fill the data which doesn't change per point
    # param = []  # will need some debugging and we probably don't care about it
    layer1_quality = []
    layer2_quality = []
    layer1_type = []
    layer2_type = []

    layer1_quality.append(f['quality'][0])
    layer2_quality.append(f['quality'][1])
    layer1_type.append(f['type'][0])
    layer2_type.append(f['type'][1])

    # initialize the data that will be loaded from the files
    gps_time = []
    layer1_id = []
    layer2_id = []
    lat = []
    lon = []
    layer1_twtt = []
    layer2_twtt = []

    # print("\n\nlayerize_h5py debug:")
    # print(f"f['lat'][:]: {f['lat'][:]}")
    # print(f"f['lat'][:]: {f['lat'][:].flatten()[0]}")

    # print(f"f['lat'][:][0]: {f['lat'][:][0]}")
    # print(f"layer1 lon: {f['lon'][:]}")

    # print("\n\n")
    already_printed = False
    # Load data from each file
    if h5py_works:
        for i in range(int(startframe), int(endframe) + 1):
            filename = data_file + str(i).zfill(3) + '.mat'
            # if h5py_works:
            with h5py.File(filename, 'r') as f:
                gps_time.append(f['gps_time'][:])
                layer1_id.append(f['id'][0])
                layer2_id.append(f['id'][1])
                lat.append(f['lat'][:].flatten())
                lon.append(f['lon'][:].flatten())
                # param.append(f['param'])
                # layer1_quality.append(f['quality'][0])
                # layer2_quality.append(f['quality'][1])

                # Extract twtt columns directly into layer1_twtt and layer2_twtt
                twtt_data = f['twtt'][:]
                layer1_twtt.extend(twtt_data[:, 0])
                layer2_twtt.extend(twtt_data[:, 1])

        # Convert lists to numpy arrays
        # debug_print(f"gps_time: {gps_time}")

        gps_time = np.concatenate(gps_time)
        lat = np.concatenate(lat)
        lon = np.concatenate(lon)
        # param = np.concatenate(param)
        layer1_twtt = np.array(layer1_twtt)
        layer2_twtt = np.array(layer2_twtt)

        # Convert lists to numpy arrays
        layer1_id = np.array(layer1_id)
        layer2_id = np.array(layer2_id)
        layer1_quality = np.array(layer1_quality)
        layer2_quality = np.array(layer2_quality)
        layer1_type = np.array(layer1_type)
        layer2_type = np.array(layer2_type)

    else:
        unflat_lat = []
        unflat_lon = []
        for i in range(int(startframe), int(endframe) + 1):
            filename = data_file + str(i).zfill(3) + '.mat'

            f = sio.loadmat(filename)
            # print the keys in f
            if already_printed:
                print(f.keys())
                already_printed = True
            for time in f['gps_time']:
                # debug_print(f"time: {time}")
                gps_time.extend(time)

            # gps_time.append(f['gps_time'][0])
            layer1_id.append(f['id'][0][0])
            # for id in f['id']:d
            # layer1_id.append(id[0][0])
            # layer2_id.append(id[0][1])
            layer2_id.append(f['id'][0][1])

            unflat_lat.append((f['lat'][0]))
            unflat_lon.append(f['lon'][0])
            # print(f"unflat_lat: {f['lat'][0]}")  # array does not have a second index, just redundant brackets

            # Extract twtt columns directly into layer1_twtt and layer2_twtt
            twtt_data = f['twtt']
            debug_print(f"len(twtt_data): {len(twtt_data)}")
            layer1_twtt.extend(twtt_data[:, 0])
            layer2_twtt.extend(twtt_data[:, 1])
        # debug_print(RED, f"gps_time: {gps_time}")

        gps_time = np.array(gps_time)
        # debug_print(f"gps_time: {gps_time}")

        # Flattening lat and lon arrays into single lists
        my_new_lat = []
        my_new_lon = []
        for arr in unflat_lat:
            lat.extend(arr)  # This will append the contents of each array into the list
        for arr in unflat_lon:
            lon.extend(arr)

    # create a layer object
    layer1 = Layer(layer1_name, gps_time, layer1_id, lat, lon, layer1_quality, layer1_twtt, layer1_type)
    layer2 = Layer(layer2_name, gps_time, layer2_id, lat, lon, layer2_quality, layer2_twtt, layer2_type)

    import time
    corrected = False
    if convert_xy:
        start = time.time()
        # Convert lat-lon to x-y
        print("Converting lat-lon to x-y...")
        for i in range(len(lat)):
            if i % 100 == 0:
                progress_bar(i, len(lat), start)
            x1, y1 = latlon_to_xy(lat[i], lon[i])
            layer1.x.append(x1)
            layer1.y.append(y1)
            layer2.x.append(x1)
            layer2.y.append(y1)

            twtt1 = layer1.twtt - layer1.twtt
            twtt2 = layer1.twtt - layer2.twtt
            layer1.twtt_corrected = twtt1
            layer2.twtt_corrected = twtt2

            depth1 = twtt_to_depth(twtt1)
            depth2 = twtt_to_depth(twtt2)
            layer1.depth = depth1
            layer2.depth = depth2

        layer1.xy_exists = True
        layer2.xy_exists = True

    # append the layer to the layers list
    layers.append(layer1)
    layers.append(layer2)
    # print(f"layer1: {layer1.layer_name} number of points: {layer1.twtt.shape[0]}")
    # print(f"layer2: {layer2.layer_name} number of points: {layer2.twtt.shape[0]}")

    if not corrected:
        for layer in layers:
            if h5py_works:
                corrected_twtt = layer.twtt - layers[0].twtt  # normalize against the surface layer
                # corrected_twtt = layer.twtt
                layer.twtt_corrected = corrected_twtt
            else:
                corrected_twtt = np.array(layer.twtt) - np.array(layers[0].twtt)
                layer.twtt_corrected = corrected_twtt

    return layers


In [4]:
# read in the layers from the layer files and save them to a pickle file
dir = f"C:\\Users\\{username}\\Documents\\cresis\\rds\\{season}\\CSARP_layer"
print(f"dir: {dir}")
folders = os.listdir(dir)
# print(f"folders: {folders}")

for folder in folders:
    flight = folder
    file_name = f"C:\\Users\\{username}\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + folder + ".pickle"
    debug_print(f"file_name: {file_name}")
    if not os.path.isfile(file_name):  # if the file does not exist
        print(f"File {file_name} does not exist. Making it...")
        mat_pickler_h5py(season, flight, testing_mode=testing)  # make it
        layers = read_layers(file_name)  # read in the layers from the pickle file
        print(f"File {file_name} created.")
    else:
        print(f"I allegedly found the file {file_name}.")
        layers = read_layers(file_name)  # read in the layers from the pickle file
        print(f"File {file_name} loaded.")

dir: C:\Users\moser\Documents\cresis\rds\2022_Antarctica_BaslerMKB\CSARP_layer
DEBUG: file_name: C:\Users\moser\Desktop\cresis_project\pickle_jar\layer_export_20221229_01.pickle
I allegedly found the file C:\Users\moser\Desktop\cresis_project\pickle_jar\layer_export_20221229_01.pickle.
Reading pickle file...
--------------------
Surface
Bottom
--------------------

File C:\Users\moser\Desktop\cresis_project\pickle_jar\layer_export_20221229_01.pickle loaded.
DEBUG: file_name: C:\Users\moser\Desktop\cresis_project\pickle_jar\layer_export_20230106_01.pickle
I allegedly found the file C:\Users\moser\Desktop\cresis_project\pickle_jar\layer_export_20230106_01.pickle.
Reading pickle file...
--------------------
Surface
Bottom
--------------------

File C:\Users\moser\Desktop\cresis_project\pickle_jar\layer_export_20230106_01.pickle loaded.
DEBUG: file_name: C:\Users\moser\Desktop\cresis_project\pickle_jar\layer_export_20230107_01.pickle
I allegedly found the file C:\Users\moser\Desktop\cresis

In [7]:
# def append_layers(first_flight, second_flight):
#     """
#     :param first_flight: a layer object
#     :param second_flight: a layer object
#     :return: a combined layer object
#     """
#     # assumes that the flights will both have a bottom and surface layer and no other relevant layers
# 
#     # print(BRIGHT_MAGENTA, section_break, RESET)
#     print("Appending layers...")
#     # append layers2 to layers1
#     # layers1 and layers2 have lists of layers. append each list with a given name to the other list with the same name if it exists. if it does not exist, throw an error
#     
#     # print(f"length of first_flight.lat: {len(first_flight[0].lat)}")
#     # print(f"length of second_flight.lat: {len(second_flight[0].lat)}")
#     # print(f"sum of lengths: {len(first_flight[0].lat) + len(second_flight[0].lat)}")
# 
#     # first_flight = layers1
#     layer_names = [layer.layer_name for layer in first_flight]
#     elevation = np.array([layer.elevation for layer in first_flight])
#     gps_time = np.array([layer.gps_time for layer in first_flight])
#     id = np.array([layer.id for layer in first_flight])
#     lat = first_flight[0].lat
#     lon = first_flight[0].lon
#     param = np.array([layer.param for layer in first_flight])
#     quality = np.array([layer.quality for layer in first_flight])
#     type = np.array([layer.type for layer in first_flight])
#     twtt = np.array([layer.twtt for layer in first_flight])
#     
#     # append the second flight
#     # if the layer names are the same, append the second flight to the first flight
#     for layer in second_flight:
#         if layer.layer_name not in layer_names:
#             raise ValueError(f"Layer {layer.layer_name} not found in first flight.")
# 
#     elevation = np.append(elevation, [layer.elevation for layer in second_flight])
#     gps_time = np.append(gps_time, [layer.gps_time for layer in second_flight])
#     # id = np.append(id, [layer.id for layer in second_flight])
#     
#     lat = np.append(lat, second_flight[0].lat)
#     lon = np.append(lon, second_flight[0].lon)
#     
#     param = np.append(param, [layer.param for layer in second_flight])
#     quality = np.append(quality, [layer.quality for layer in second_flight])
#     type = np.append(type, [layer.type for layer in second_flight])
#     twtt = np.append(twtt, [layer.twtt for layer in second_flight])
#     
#     # print(f"length of id: {len(id)}")
#     # print(f"length of lat: {len(lat)}")
#     # print(section_break)
#     
#     print(BRIGHT_MAGENTA, section_break, RESET, "\n")
#     # return first_flight
#     
#     surface_layer = Layer(layer_names[0], gps_time, id, lat, lon, quality, type[0], twtt[0])
#     bottom_layer = Layer(layer_names[1], gps_time, id, lat, lon, quality, type[1], twtt[1])
#     
#     combined_flights = [surface_layer, bottom_layer]
#     
#     return combined_flights


def append_layers(first_flight, second_flight):
    """
    :param first_flight: a layer object
    :param second_flight: a layer object
    :return: a combined layer object
    """
    # assumes that the flights will both have a bottom and surface layer and no other relevant layers

    # print(BRIGHT_MAGENTA, section_break, RESET)
    print("Appending layers...")
    # append layers2 to layers1
    # layers1 and layers2 have lists of layers. append each list with a given name to the other list with the same name if it exists. if it does not exist, throw an error
    
    # print(f"length of first_flight.lat: {len(first_flight[0].lat)}")
    # print(f"length of second_flight.lat: {len(second_flight[0].lat)}")
    # print(f"sum of lengths: {len(first_flight[0].lat) + len(second_flight[0].lat)}")

    # first_flight = layers1
    surface_1 = first_flight[0]
    bottom_1 = first_flight[1]
    
    print(f"layer_name0: {surface_1.layer_name}")
    print(f"layer_name1: {bottom_1.layer_name}")
    
    layer_names = [surface_1.layer_name, bottom_1.layer_name]
    elevation = np.array(surface_1.elevation)
    gps_time = np.array(surface_1.gps_time)
    # id = np.array([layer.id for layer in first_flight])
    id = surface_1.id
    lat = surface_1.lat
    lon = surface_1.lon
    param = np.array([layer.param for layer in first_flight]) # probably None
    # quality = np.array([layer.quality for layer in first_flight])
    quality = surface_1.quality[0]
    type = surface_1.type
    twtt0 = np.array(surface_1.twtt)
    twtt1 = np.array(bottom_1.twtt)
    
    # append the second flight
    # if the layer names are the same, append the second flight to the first flight
    for layer in second_flight:
        if layer.layer_name not in layer_names:
            raise ValueError(f"Layer {layer.layer_name} not found in first flight.")

    surface_2 = second_flight[0]
    bottom_2 = second_flight[1]
    
    elevation = np.append(elevation, surface_2.elevation)
    # gps_time = np.append(gps_time, [layer.gps_time for layer in second_flight])
    gps_time = np.append(gps_time, surface_2.gps_time)
    # id = np.append(id, [layer.id for layer in second_flight])
    
    lat = np.append(lat, surface_2.lat)
    lon = np.append(lon, surface_2.lon)
    
    param = np.append(param, [layer.param for layer in second_flight])
    quality = np.append(quality, [layer.quality for layer in second_flight])
    type = np.append(type, [layer.type for layer in second_flight])
    # twtt = np.append(twtt, [layer.twtt for layer in second_flight])
    twtt0 = np.append(twtt0, surface_2.twtt)
    twtt1 = np.append(twtt1, bottom_2.twtt)
    
    # print(f"length of id: {len(id)}")
    # print(f"length of lat: {len(lat)}")
    # print(section_break)
    
    print(BRIGHT_MAGENTA, section_break, RESET, "\n")
    # return first_flight
    
    surface_layer = Layer(layer_names[0], gps_time, id, lat, lon, quality, twtt0, type[0])
    bottom_layer = Layer(layer_names[1], gps_time, id, lat, lon, quality, twtt1, type[1])
    
    combined_flights = [surface_layer, bottom_layer]
    
    return combined_flights

In [16]:
dir = "C:\\Users\\moser\\Documents\\cresis\\rds\\" + season + "\\CSARP_layer"
# print(f"dir: {dir}")
folders = os.listdir(dir)

layers = read_layers("C:\\Users\\moser\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + folders[0] + ".pickle", quiet=True)
# print(f"len(layers): {len(layers)}")

tot_len = 0

for folder in folders[0:3]:
    if folder == folders[0]:
        tot_len += len(layers[0].lat)
        continue
    print(BRIGHT_MAGENTA, section_break, RESET, "\nfolder: ", folder)
    borked = mat_pickler_h5py(season, folder, testing_mode=testing)  # make it
    if borked:
        debug_print(BRIGHT_RED, f"Folder {folder} is borked. Skipping it.")
        borked = False
        continue        
    else:
        # print(f"folder: {folder}")
        layer = read_layers("C:\\Users\\moser\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + folder + ".pickle", quiet=True)
        layer_len = len(layer[0].lat)
        # print(f"len(layer): {layer_len}")
        tot_len += layer_len
    layers = append_layers(layers, layer)
# print(f"total length: {tot_len}")

# print the length of each member of the layers[0] object
print(f"length of name array: {len(layers[0].layer_name)}")
# print(f"length of elevation array: {len(layers[0].elevation)}")
print(f"length of gps_time array: {len(layers[0].gps_time)}")
print(f"length of id array: {len(layers[0].id)}")
print(f"length of lat array: {len(layers[0].lat)}")
print(f"length of lon array: {len(layers[0].lon)}")
# print(f"length of param array: {len(layers[0].param)}")
print(f"length of quality array: {len(layers[0].quality)}")
# print(layers[0].twtt)
print(f"length of twtt array: {len(layers[0].twtt)}")
# print(f"length of twtt_corrected array: {len(layers[0].twtt_corrected)}")
print(f"type array: {layers[0].type}")
print(f"length of x array: {len(layers[0].x)}")
print(f"length of y array: {len(layers[0].y)}")
# print(f"length of xy_exists array: {len(layers[0].xy_exists)}")
print(f"length of depth array: {len(layers[0].depth)}")


 --------------------  
folder:  20230106_01
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
Appending layers...
layer_name0: Surface
layer_name1: Bottom
 --------------------  

 --------------------  
folder:  20230107_01
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2

In [ ]:
print(f"length of lat array: {len(layers[0].lat)}")
print(f"layers[0].twtt: {layers[0].twtt}")

In [ ]:
directory = os.getcwd() + "\\pickle_jar\\"
# file_name = directory + "layer_export" + attributes_file[5:-4] + ".pickle"
file_name = directory + "layer_export_" + season + "_full" + ".pickle"
pickle.dump(layers, open(file_name, "wb"))

In [14]:
file_name = f"C:\\Users\\{username}\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + folders[0] + ".pickle"
tl = read_layers(file_name)
# print(f"id: {tl[0].id}")
print(f"param: {tl[0].param}")
# print(f"quality: {tl[0].quality}")
# print(f"type: {tl[0].type}")
print(f"len(twtt): {len(tl[0].twtt)}")
print(f"len(gpstime): {len(tl[0].gps_time)}")

Reading pickle file...
--------------------
Surface
Bottom
--------------------

param: None
len(twtt): 56
len(gpstime): 92881


In [30]:
dir = ('C:\\Users\\moser\\Documents\\cresis\\rds\\' + season + '\\CSARP_layer\\' + flight + '\\')
# print(f"dir: {dir}")
folders = os.listdir(dir)
data_file = dir + 'Data_' + flight + '_'
attributes_file = dir + 'layer_' + flight
# attributes_file = dir + '\\layer_' + flight + "\\"

layers = []
files = os.listdir(dir)  # list of files in the directory
# TODO: shift to differentiating data vs layer based on file name. Data starts with "Data_" and layer starts with "layer_"
startframe = '001'  # the first file number in the directory
endframe = str(len(files) - 1).zfill(3)  # the number of files in the directory
# debug_print(f"attribute_file: {attribute_file}")
attribute_mat = h5py.File(attributes_file + '.mat', 'r')
# print the keys in the attribute file

decimal1 = attribute_mat[np.array([attribute_mat['lyr_name'][0][0]])[0]]
decimal1_name = [decimal1[i][0] for i in range(len(decimal1))]  # a list of unicode values of the characters
decimal2 = attribute_mat[np.array([attribute_mat['lyr_name'][1][0]])[0]]
decimal2_name = [decimal2[i][0] for i in range(len(decimal2))]  # a list of unicode values of the characters
name1 = ''.join(chr(i) for i in decimal1_name)  # convert the unicode values to a string
name2 = ''.join(chr(i) for i in decimal2_name)  # convert the unicode values to a string

layer1_name = name1
layer2_name = name2

# print(f"data_file: {data_file}")
filled_data_file = data_file + str(1).zfill(3) + '.mat'
# print(f"filled_data_file: {filled_data_file}")

h5py_works = True
# debug_print("made it to layerize_h5py", "passed the filled_data_file variable to h5py.File",
# "\n", "filled_data_file: ", filled_data_file, "\n")
try:
    f = h5py.File(filled_data_file, 'r')
    debug_print(BRIGHT_GREEN, f"Opened {filled_data_file} with h5py")
except:
    # debug_print(RED, f"Error: could not open {filled_data_file} with h5py, ignoring this file")
    # return True
    # print(f"Error: could not open {filled_data_file} with h5py, attempting with sio.loadmat")
    try:
        f = sio.loadmat(filled_data_file)
        h5py_works = False
    except:
        debug_print(RED, f"Error: could not open {filled_data_file} with sio.loadmat, exiting")
        # sys.exit(1)
        # return True      


# f = h5py.File(data_file + str(3).zfill(3) + '.mat', 'r')  #
# print the contents of the 'param' key
# print(f"f['param']: {f['param']}")
# print(f"f['param'].keys(): {list(f['param'].keys())}")

filename = data_file + '001.mat'  # fill the data which doesn't change per point
# param = []  # will need some debugging and we probably don't care about it
layer1_quality = []
layer2_quality = []
layer1_type = []
layer2_type = []

layer1_quality.append(f['quality'][0])
layer2_quality.append(f['quality'][1])
layer1_type.append(f['type'][0])
layer2_type.append(f['type'][1])

# initialize the data that will be loaded from the files
gps_time = []
layer1_id = []
layer2_id = []
lat = []
lon = []
layer1_twtt = []
layer2_twtt = []

# print("\n\nlayerize_h5py debug:")
# print(f"f['lat'][:]: {f['lat'][:]}")
# print(f"f['lat'][:]: {f['lat'][:].flatten()[0]}")

# print(f"f['lat'][:][0]: {f['lat'][:][0]}")
# print(f"layer1 lon: {f['lon'][:]}")

# print("\n\n")
already_printed = False
# Load data from each file
if h5py_works:
    for i in range(int(startframe), int(endframe) + 1):
        filename = data_file + str(i).zfill(3) + '.mat'
        # if h5py_works:
        with h5py.File(filename, 'r') as f:
            gps_time.append(f['gps_time'][:])
            layer1_id.append(f['id'][0])
            layer2_id.append(f['id'][1])
            lat.append(f['lat'][:].flatten())
            lon.append(f['lon'][:].flatten())
            # param.append(f['param'])
            # layer1_quality.append(f['quality'][0])
            # layer2_quality.append(f['quality'][1])

            # Extract twtt columns directly into layer1_twtt and layer2_twtt
            twtt_data = f['twtt'][:]
            layer1_twtt.extend(twtt_data[:, 0])
            layer2_twtt.extend(twtt_data[:, 1])

    # Convert lists to numpy arrays
    # debug_print(f"gps_time: {gps_time}")

    gps_time = np.concatenate(gps_time)
    lat = np.concatenate(lat)
    lon = np.concatenate(lon)
    # param = np.concatenate(param)
    layer1_twtt = np.array(layer1_twtt)
    layer2_twtt = np.array(layer2_twtt)

    # Convert lists to numpy arrays
    layer1_id = np.array(layer1_id)
    layer2_id = np.array(layer2_id)
    layer1_quality = np.array(layer1_quality)
    layer2_quality = np.array(layer2_quality)
    layer1_type = np.array(layer1_type)
    layer2_type = np.array(layer2_type)

else:
    unflat_lat = []
    unflat_lon = []
    for i in range(int(startframe), int(endframe) + 1):
        filename = data_file + str(i).zfill(3) + '.mat'

        f = sio.loadmat(filename)
        # print the keys in f
        if already_printed:
            print(f.keys())
            already_printed = True
        for time in f['gps_time']:
            # debug_print(f"time: {time}")
            gps_time.extend(time)

        # gps_time.append(f['gps_time'][0])
        layer1_id.append(f['id'][0][0])
        # for id in f['id']:d
        # layer1_id.append(id[0][0])
        # layer2_id.append(id[0][1])
        layer2_id.append(f['id'][0][1])

        unflat_lat.append((f['lat'][0]))
        unflat_lon.append(f['lon'][0])
        # print(f"unflat_lat: {f['lat'][0]}")  # array does not have a second index, just redundant brackets

        # Extract twtt columns directly into layer1_twtt and layer2_twtt
        twtt_data = f['twtt']
        debug_print(f"len(twtt_data): {len(twtt_data)}")
        layer1_twtt.extend(twtt_data[:, 0])
        layer2_twtt.extend(twtt_data[:, 1])
    # debug_print(RED, f"gps_time: {gps_time}")

    gps_time = np.array(gps_time)
    # debug_print(f"gps_time: {gps_time}")

    # Flattening lat and lon arrays into single lists
    my_new_lat = []
    my_new_lon = []
    for arr in unflat_lat:
        lat.extend(arr)  # This will append the contents of each array into the list
    for arr in unflat_lon:
        lon.extend(arr)

# create a layer object
layer1 = Layer(layer1_name, gps_time, layer1_id, lat, lon, layer1_quality, layer1_twtt, layer1_type)
layer2 = Layer(layer2_name, gps_time, layer2_id, lat, lon, layer2_quality, layer2_twtt, layer2_type)

import time
corrected = False
if convert_xy:
    start = time.time()
    # Convert lat-lon to x-y
    print("Converting lat-lon to x-y...")
    for i in range(len(lat)):
        if i % 100 == 0:
            progress_bar(i, len(lat), start)
        x1, y1 = latlon_to_xy(lat[i], lon[i])
        layer1.x.append(x1)
        layer1.y.append(y1)
        layer2.x.append(x1)
        layer2.y.append(y1)

        twtt1 = layer1.twtt - layer1.twtt
        twtt2 = layer1.twtt - layer2.twtt
        layer1.twtt_corrected = twtt1
        layer2.twtt_corrected = twtt2

        depth1 = twtt_to_depth(twtt1)
        depth2 = twtt_to_depth(twtt2)
        layer1.depth = depth1
        layer2.depth = depth2

    layer1.xy_exists = True
    layer2.xy_exists = True

# append the layer to the layers list
layers.append(layer1)
layers.append(layer2)
# print(f"layer1: {layer1.layer_name} number of points: {layer1.twtt.shape[0]}")
# print(f"layer2: {layer2.layer_name} number of points: {layer2.twtt.shape[0]}")

if not corrected:
    for layer in layers:
        if h5py_works:
            corrected_twtt = layer.twtt - layers[0].twtt  # normalize against the surface layer
            # corrected_twtt = layer.twtt
            layer.twtt_corrected = corrected_twtt
        else:
            corrected_twtt = np.array(layer.twtt) - np.array(layers[0].twtt)
            layer.twtt_corrected = corrected_twtt

DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2
DEBUG: len(twtt_data): 2


NameError: name 'convert_xy' is not defined